This notebook contains all the BERT models we used durinf our project, in order to use it properly, run the initialisation part and then the BERT you want to use. We advise to restart the run time after each model to avoid any problem.

# Initialisation


In [ ]:
!pip install datasets transformers huggingface_hub sacremoses
!apt-get install git-lfs
!pip install transformers[torch]
!pip install accelerate -U
!pip install torch sklearn


# Import required packages
from transformers import pipeline, DataCollatorWithPadding
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from google.colab import files
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from google.colab import files
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
import pandas as pd
import numpy as np
import torch
import xgboost as xgb
import torchgen
import string
import nltk
import torch
import xgboost as xgb


torch.cuda.is_available()

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


True

In [ ]:
# URL to the raw CSV file on GitHub
url = 'https://raw.githubusercontent.com/msperand/Machine_Learning_Project/main/Data/sample_submission.csv'

# Read the CSV file into a DataFrame
sample_submission = pd.read_csv(url)

# Display the first few rows of the DataFrame
print(sample_submission)


        id difficulty
0        0         A1
1        1         A1
2        2         A1
3        3         A1
4        4         A1
...    ...        ...
1195  1195         A1
1196  1196         A1
1197  1197         A1
1198  1198         A1
1199  1199         A1

[1200 rows x 2 columns]


In [ ]:
# URL to the raw CSV file on GitHub
url2 = 'https://raw.github.com/msperand/Machine_Learning_Project/main/Data/training_data.csv'

# Read the CSV file into a DataFrame
training_data = pd.read_csv(url2)

# Display the first few rows of the DataFrame
print(training_data)

        id                                           sentence difficulty
0        0  Les coûts kilométriques réels peuvent diverger...         C1
1        1  Le bleu, c'est ma couleur préférée mais je n'a...         A1
2        2  Le test de niveau en français est sur le site ...         A1
3        3           Est-ce que ton mari est aussi de Boston?         A1
4        4  Dans les écoles de commerce, dans les couloirs...         B1
...    ...                                                ...        ...
4795  4795  C'est pourquoi, il décida de remplacer les hab...         B2
4796  4796  Il avait une de ces pâleurs splendides qui don...         C1
4797  4797  Et le premier samedi de chaque mois, venez ren...         A2
4798  4798  Les coûts liés à la journalisation n'étant pas...         C2
4799  4799  Sur le sable, la mer haletait de toute la resp...         C2

[4800 rows x 3 columns]


In [ ]:
# URL to the raw CSV file on GitHub
url3 = 'https://raw.github.com/msperand/Machine_Learning_Project/main/Data/unlabelled_test_data.csv'

# Read the CSV file into a DataFrame
test_data = pd.read_csv(url3)

# Display the first few rows of the DataFrame
print(test_data)

        id                                           sentence
0        0  Nous dûmes nous excuser des propos que nous eû...
1        1  Vous ne pouvez pas savoir le plaisir que j'ai ...
2        2  Et, paradoxalement, boire froid n'est pas la b...
3        3  Ce n'est pas étonnant, car c'est une saison my...
4        4  Le corps de Golo lui-même, d'une essence aussi...
...    ...                                                ...
1195  1195  C'est un phénomène qui trouve une accélération...
1196  1196  Je vais parler au serveur et voir si on peut d...
1197  1197  Il n'était pas comme tant de gens qui par pare...
1198  1198      Ils deviennent dangereux pour notre économie.
1199  1199  Son succès a généré beaucoup de réactions néga...

[1200 rows x 2 columns]


# DistilBERT

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd

# Split data into training and validation sets
train_data, val_data = train_test_split(training_data, test_size=0.1)

# Assuming 'sentence' is your text column and 'difficulty' is the target
train_texts = train_data['sentence'].tolist()
train_labels = train_data['difficulty'].tolist()
val_texts = val_data['sentence'].tolist()
val_labels = val_data['difficulty'].tolist()


In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_labels)
val_labels = label_encoder.transform(val_labels)


In [ ]:
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

def tokenize_data(texts):
    return tokenizer(texts, padding=True, truncation=True, max_length=128, return_tensors="pt")

train_encodings = tokenize_data(train_texts)
val_encodings = tokenize_data(val_texts)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = TextDataset(train_encodings, train_labels)
val_dataset = TextDataset(val_encodings, val_labels)


In [ ]:
from transformers import Trainer, TrainingArguments, DistilBertForSequenceClassification

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # number of training epochs
    per_device_train_batch_size=8,   # batch size for training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    evaluation_strategy="epoch"
)

model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(label_encoder.classes_))

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
trainer.train()


Epoch,Training Loss,Validation Loss
1,1.316200,1.327773
2,1.048400,1.187553
3,0.836000,1.157645


TrainOutput(global_step=1620, training_loss=1.1434333630550055, metrics={'train_runtime': 181.333, 'train_samples_per_second': 71.471, 'train_steps_per_second': 8.934, 'total_flos': 429224987934720.0, 'train_loss': 1.1434333630550055, 'epoch': 3.0})

In [ ]:
model.save_pretrained('./saved_model')
tokenizer.save_pretrained('./saved_model')


('./saved_model/tokenizer_config.json',
 './saved_model/special_tokens_map.json',
 './saved_model/vocab.txt',
 './saved_model/added_tokens.json')

In [ ]:
import pandas as pd
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from torch.utils.data import DataLoader
import numpy as np

# Assuming the tokenizer and model are already loaded
tokenizer = DistilBertTokenizer.from_pretrained('./saved_model')
model = DistilBertForSequenceClassification.from_pretrained('./saved_model')
model.eval()  # Set the model to evaluation mode

# Load test data
test_texts = test_data['sentence'].tolist()  # Assuming 'sentence' column exists

# Tokenize test data
def tokenize_data(texts):
    return tokenizer(texts, padding=True, truncation=True, max_length=128, return_tensors="pt")

test_encodings = tokenize_data(test_texts)

# Create DataLoader
class TestData(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings['input_ids'])

test_dataset = TestData(test_encodings)
test_loader = DataLoader(test_dataset, batch_size=32)

# Predicting
all_predictions = []
with torch.no_grad():
    for batch in test_loader:
        inputs = {k: v.to(model.device) for k, v in batch.items()}
        outputs = model(**inputs)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        all_predictions.extend(predictions.cpu().numpy())

# Convert numerical predictions back to labels
predicted_labels = label_encoder.inverse_transform(all_predictions)

# Prepare DataFrame for submission
submission_df = pd.DataFrame({
    'id': test_data['id'],  # Make sure 'id' column matches the test data
    'difficulty': predicted_labels
})


<ipython-input-13-600a8d7cac23>:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


In [ ]:
# Save the updated DataFrame to a CSV file
file_name = 'sample_submission_Bert_FT.csv'  # Temporary file name in the Colab environment
submission_df.to_csv(file_name, index=False)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# bert-base-multilingual-cased

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, get_scheduler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import pandas as pd
import torch

# Assuming the data is loaded into DataFrame 'training_data'
data_subset = training_data

# Split the subset into training and validation sets
train_data, val_data = train_test_split(data_subset, test_size=0.2, random_state=42)

# Encode the labels
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_data['difficulty'])
val_labels = label_encoder.transform(val_data['difficulty'])

# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# Function to tokenize the data
def tokenize_function(texts):
    return tokenizer(texts, padding="max_length", truncation=True, max_length=128, return_tensors="pt")

# Tokenize the data
train_encodings = tokenize_function(train_data['sentence'].tolist())
val_encodings = tokenize_function(val_data['sentence'].tolist())

# Prepare datasets
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = Dataset(train_encodings, train_labels)
val_dataset = Dataset(val_encodings, val_labels)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

# Load the model
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=len(label_encoder.classes_))


# Training arguments
""" Baseline Version
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=4,  # Increased epochs
    per_device_train_batch_size=16,  # Adjusted batch size
    per_device_eval_batch_size=32,  # Adjusted eval batch size
    warmup_steps=300,  # Adjusted warmup steps
    weight_decay=0.05,  # Increased weight decay
    logging_dir='./logs',
    logging_steps=50,  # More frequent logging
    evaluation_strategy="steps",
    eval_steps=100,  # More frequent evaluation
    lr_scheduler_type='linear',  # Learning rate scheduler
    load_best_model_at_end=True,  # Load the best model at the end of training
    metric_for_best_model='accuracy'  # Optimize for accuracy
)"""
# New Version 2
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,              # Consider reducing if overfitting
    per_device_train_batch_size=16,  # Adjust batch size
    per_device_eval_batch_size=32,
    warmup_steps=300,
    weight_decay=0.05,               # Increased weight decay
    logging_dir='./logs',
    logging_steps=50,
    eval_steps=100,                  # More frequent evaluation
    evaluation_strategy="steps",
    load_best_model_at_end=True,
    metric_for_best_model='f1'       # Assuming F1 is a more relevant metric
)


# Trainer setup with a custom optimizer and scheduler
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics  # Define a function to compute metrics if needed
)

trainer.train()

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-15-9ac736fefa52>:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
100,1.563900,1.363547,0.419792,0.414095,0.419792,0.385712
200,1.223500,1.200585,0.482292,0.484447,0.482292,0.470343
300,1.119500,1.115899,0.532292,0.538889,0.532292,0.524729
400,1.140900,1.054127,0.563542,0.567206,0.563542,0.561861
500,0.932500,1.136017,0.530208,0.552440,0.530208,0.524202
600,0.698700,1.144179,0.541667,0.559436,0.541667,0.541428
700,0.671600,1.110075,0.555208,0.565522,0.555208,0.553879


<ipython-input-15-9ac736fefa52>:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-15-9ac736fefa52>:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-15-9ac736fefa52>:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-15-9ac736fefa52>:37: UserWarning: To copy construct fro

TrainOutput(global_step=720, training_loss=1.0760213494300843, metrics={'train_runtime': 334.4033, 'train_samples_per_second': 34.449, 'train_steps_per_second': 2.153, 'total_flos': 757787053916160.0, 'train_loss': 1.0760213494300843, 'epoch': 3.0})

In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
import numpy as np

# Assuming the model and tokenizer are already loaded
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
#model = BertForSequenceClassification.from_pretrained('./model_directory', num_labels=len(label_encoder.classes_))
model.eval()  # Set the model to evaluation mode

# Load the test data (assuming it's loaded into a DataFrame named test_data)
# test_data = pd.read_csv('path/to/your/test_data.csv')  # Uncomment if you need to load from a file

# Tokenize the test data
def tokenize_data(texts):
    return tokenizer(texts, padding=True, truncation=True, max_length=128, return_tensors="pt")

test_encodings = tokenize_data(test_data['sentence'].tolist())

# Create a DataLoader for the test data
class TestData(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings['input_ids'])

test_dataset = TestData(test_encodings)
test_loader = DataLoader(test_dataset, batch_size=32)

# Predicting with the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
predictions = []

with torch.no_grad():
    for batch in test_loader:
        inputs = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**inputs)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=-1)
        predictions.extend(preds.cpu().numpy())

# Convert numerical predictions back to labels
predicted_labels = label_encoder.inverse_transform(predictions)

# Create DataFrame with IDs and predicted difficulties
submission_df = pd.DataFrame({'id': test_data['id'], 'difficulty': predicted_labels})

# Save predictions to a CSV file
submission_file_path = 'BERT_base_full_New_version_2.csv'
submission_df.to_csv(submission_file_path, index=False)

# Provide a download link in Google Colab
from google.colab import files
files.download(submission_file_path)


<ipython-input-16-ef04eaa918c0>:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# CamemBERT base

In [ ]:
from transformers import CamembertTokenizer, CamembertForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
# Assuming training_data is your dataset
data_subset = training_data

# Split the data into training and validation sets
train_data, val_data = train_test_split(data_subset, test_size=0.2, random_state=11)

# Encode the labels
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_data['difficulty'])
val_labels = label_encoder.transform(val_data['difficulty'])

# Initialize tokenizer and model
tokenizer = CamembertTokenizer.from_pretrained('camembert-base')

# Load the model for sequence classification
model = CamembertForSequenceClassification.from_pretrained('camembert-base', num_labels=len(label_encoder.classes_))

# Function to tokenize the data
def tokenize_function(texts):
    return tokenizer(texts, padding="max_length", truncation=True, max_length=128, return_tensors="pt")

# Tokenize the data
train_encodings = tokenize_function(train_data['sentence'].tolist())
val_encodings = tokenize_function(val_data['sentence'].tolist())

# Prepare datasets
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = Dataset(train_encodings, train_labels)
val_dataset = Dataset(val_encodings, val_labels)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    warmup_steps=300,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

<ipython-input-6-0c1044ef59d3>:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.224000,1.065201,0.532292,0.537756,0.532292,0.512948
2,1.030200,1.114779,0.523958,0.549485,0.523958,0.508219
3,0.744600,1.119856,0.565625,0.582594,0.565625,0.567509
4,0.417300,1.285190,0.578125,0.620061,0.578125,0.579117
5,0.268600,1.455418,0.579167,0.614417,0.579167,0.579901


<ipython-input-6-0c1044ef59d3>:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-6-0c1044ef59d3>:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-6-0c1044ef59d3>:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-6-0c1044ef59d3>:35: UserWarning: To copy construct from a 

TrainOutput(global_step=2400, training_loss=0.7771795357763768, metrics={'train_runtime': 194.3695, 'train_samples_per_second': 98.781, 'train_steps_per_second': 12.348, 'total_flos': 1262978423193600.0, 'train_loss': 0.7771795357763768, 'epoch': 5.0})

In [ ]:
from torch.utils.data import DataLoader, Dataset

model.eval()  # Set the model to evaluation mode

# Load the test data (assuming it's loaded into a DataFrame named test_data)
# test_data = pd.read_csv('path/to/your/test_data.csv')  # Uncomment if you need to load from a file

# Tokenize the test data
def tokenize_data(texts):
    return tokenizer(texts, padding=True, truncation=True, max_length=128, return_tensors="pt")

test_encodings = tokenize_data(test_data['sentence'].tolist())

# Create a DataLoader for the test data
class TestData(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings['input_ids'])

test_dataset = TestData(test_encodings)
test_loader = DataLoader(test_dataset, batch_size=32)

# Predicting with the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
predictions = []

with torch.no_grad():
    for batch in test_loader:
        inputs = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**inputs)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=-1)
        predictions.extend(preds.cpu().numpy())

# Convert numerical predictions back to labels
predicted_labels = label_encoder.inverse_transform(predictions)

# Create DataFrame with IDs and predicted difficulties
submission_df = pd.DataFrame({'id': test_data['id'], 'difficulty': predicted_labels})

# Save predictions to a CSV file
submission_file_path = 'CamemBERT_base_test_5epochs_rs11.csv'
submission_df.to_csv(submission_file_path, index=False)

# Provide a download link in Google Colab
from google.colab import files
files.download(submission_file_path)

<ipython-input-7-7321fe3d03c8>:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>